In [ ]:
#————————————————————

# Name: Azure OpenAI API, Fine-Tuning

# Purpose: This notebook will use the Fine-tuned to create vegan recipes.

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch), Alex Dean (adean@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 25.05.2024
# Python Version: 3.10.4

# Additionals:
# Use Region EASTUS2

# If necessary, download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [4]:
# Import Python packages
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import json
from IPython.display import clear_output
from IPython.core.display import HTML
import requests
import random

In [5]:
# Load required variables from .env file.
load_dotenv(dotenv_path=Path("/workspaces/azure-openai-lab/.venv/.env")) #Error sometimes due to \ or \\. Try one or the other. "C:\\Python\\azure-openai-lab\\.venv\\.env"

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [6]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    azure_endpoint = azure_oai_endpoint,
    api_version = "2024-02-15-preview"
    )

In [7]:
# Zero-Shot learning. Model has a token limit of 4096.

# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as a head chef such as Joël Robuchon who specializes in simple contemporary cuisine.
Action: Create well-thought-out and flavourful vegan recipes from a list of ingredients implementing classic culinary techniques.
Target Audience: The recipients of these vegan recipes are couples who want to cook a special meal at least once a week.

---

### OUTPUT FORMAT
Output only one vegan recipe and return it as a JSON object with the following format:
{"name":"","minutes":,"tags":"[]","nutrition":"[]","n_steps":"","steps":"[]","description":"","ingredients":"[]", "n_ingredients":}

The variables should contain the following information:
- name: the name of the recipe.
- minutes: the time in minutes to prepare the recipe.
- tags: a list of words that characterize the recipe.
- nutrition: a list of numeric values representing calories, total fat, sugar, sodium, protein, saturated fat, and carbohydrates.
- n_steps: the number of steps to prepare the recipe.
- steps: a list of steps to prepare the recipe.
- description: a summary of the recipe.
- ingredients: a list of the ingredient names in the recipe.
- n_ingredients: the total number of ingredients used in the recipe.
"""

# Create a prompt of ingredients the model should create a recipe from
ingredients = """'veal roast', 'butter', 'oil', 'carrots', 'onions', 'parsley sprigs', 'bay leaf', 'thyme', 'salt', 'pepper', 'bacon'"""

# Send request to Azure OpenAI model
completion = client.chat.completions.create(
  model = "gpt-35-turbo-0613-ft",
  #response_format={ "type": "json_object" }, # Not support for fine tuned models
  messages = [    
    {"role": "system", "content": systemcontent},
    {"role": "user", "content": ingredients}
  ]
)

# View generated recipe
result = completion.choices[0].message.content
print(result)


{"name":"vegan instant pot sunday supper  mock \"veal\" roast with brown butter and lemon", "minutes":125, "tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'main-dish', 'beans', 'vegetables', 'italian', 'vegan', 'vegetarian', 'potatoes', 'dietary', 'one-dish-meals', 'soy-tofu', 'pressure-cooker', '3-steps-or-less', '4-hours-or-less']","nutrition":"[512.2, 25.0, 49.0, 684.0, 47.0, 14.0, 14.0]","n_steps":20,"steps":"['put some holes on the roast all around and stud with 1\/2 the garlic', \"marinade- gently heat butter in a small skillet over medium?low flame until it begins to brown and smell nutty .\", 'you really have to watch this carefully so it does not burn', 'squeeze lemon into a 4 cup or larger liquid measure , add water to equal 4 cups , and set aside', 'pour olive oil in the bottom of the instant pot and put the roast in ', 'add the soy sauce , sprinkle on the herbs , spread with the remaining garlic , and salt and pepper 

In [9]:
#Transform output to pandas dataframe and save as CSV file

# Clean up Azure OpenAI Output
json_data = result.strip('` \n')

if json_data.startswith('json'):
    json_data = json_data[4:]  # Remove the first 4 characters 'json'

recipes_from_ft_json = json.loads(json_data)
recipes_from_ft = pd.json_normalize(recipes_from_ft_json)
# path_output = r"/workspaces/azure-openai-lab/data/recipes-from-ft.csv" #r"C:\Python\azure-openai-lab\data\recipes-from-ft.csv"
# recipes_from_ft.to_csv(path_output, sep='\t', encoding='utf-8', index=False)